# Kmart Analysis

## Objectives

*   Collect and arrange raw dataset from GitHub repo
*   Process data (handle missing data, duplicates, junk data, engineer) for subsequent analysis
* Generate pipeline to process the data
* Generate dataset for subsequent analysis


## Inputs

* Set of CSV data from GitHub repo

## Outputs

* Cleaned and engineered dataset for subsequent analysis (the notebook was run on Colab, the data was downloaded manually and updated manually to the GitHub repo)
* Pipeline to process the data that can be handled to a software engineer

## Roadmap

* Liase with data engineering to better understand data generation mechanism (not critical, but worth to add)





---

# Install and load packages

In [ ]:
! pip install pandas==1.3.5
! pip install matplotlib==3.5.0
! pip install seaborn==0.11.2
! pip install plotly==5.1.0
! pip install feature-engine==1.4.0 
! pip install scikit-learn==1.1.1
! pip install pandas-profiling==3.3.0 

import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings('ignore')

---

# Load Data

Collect data from GitHub repo
* **NOTE: we don't have clarity on the data generation mechanism**

In [ ]:
df_raw = pd.DataFrame()
for month in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',  'December']:
  df_raw = df_raw.append( pd.read_csv(f"https://raw.githubusercontent.com/jsc1535/K-Mart-Data-Analysis/main/Sales_{month}_2019.csv") )

df_raw.reset_index(inplace=True, drop=True)
df = df_raw.copy()

print(df_raw.shape)
df_raw.head()

Data Documentation
* There is no official data documentation, however Kmart is an online retailer in the US.
* The understanding is that:
  * The dataset shows **sales levels in 2019**. The sales results is made by the composition of its annual customer online orders.
  * The data is broke down by product, order date, order id, quantity, unitary price (likely in US$), and customer address
  * **Each row** is a product bought in a given order. A customer order is represented by a set of Order ID 

---

# Quick EDA

Check and handle:
* Missing data
* Duplicates
* Junk Data

### Missing data

* There are missing data, but it is small (545) compared to the total rows (186k) - 0.28%

In [ ]:
missing_data = df.isna().sum().sort_values(ascending=False) 
pd.DataFrame(data= {"Absolute Levels": missing_data,
                    "Relative Levels (%)": round(missing_data / len(df) * 100 , 2)
    })

Inspect missing data dataframe
* It looks that missing values happen in all columns in a set of rows


In [ ]:
df_na = df[df.isna().any(axis=1)].copy()
df_na.head(3)

Missing values happen in all columns in a set of rows
* it is 0 non-null in all columns



In [ ]:
df_na.info()

Just as an exploratory exercise: in which months has it happened?
* Assumption: We are taking the indices from `df_na` and substracting 1, so we can get predecessor datapoint, which hopefully doesn't have missing data. 
* If predecessor has missing data, we iterate on getting its predecessor
* By using `.isna().sum()`, we not there is not missing data in the predecessor

In [ ]:
df.loc[df_na.index - 1].isna().sum()

Let's filter date on `df.loc[df_na.index - 1]` , extract month and plot its distribution
* The missing values happen across all months
* There is 'or' level for month, which looks to be junk data. We will ignore that for a moment, since this data comes from the missing values predecessor. We will explore junk data in a moment 

In [ ]:
df_date_na = df.loc[df_na.index - 1].filter(['Order Date'], axis=1)
df_date_na['Month'] = df_date_na['Order Date'].apply(lambda x: x[:2]) # extracts month from pattern mm/dd/yy HH:MM	

sns.countplot(data=df_date_na,x='Month')
plt.show()
del df_date_na, df_na

So what? The analysis indicates: 
* For some reason there is missing data happening all over the year. In the workplace, this information would be taken to the data engineering team, so it can be further investigated the root causes and potential solutions
* **For our project purpose, we will remove these missing data**

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

### Duplicates

* We note what looks to be junk data (like index 1102) and real duplicates, we will first manage junk data, then get back to duplicates
* A junk row looks to be "all junked", meaning all columns there are junk

In [ ]:
df[df.duplicated()] 

### Junk Data

Quick EDA with Pandas Profiling
* No missing data, as expected
* Junk data on `Order ID, Quanrtity Ordered, Order Date, Purchase Address`
* We will explore other variables distribution in detail in another moment


In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

Get rid of junk data

In [ ]:
df = df.query('`Order ID` != "Order ID"') 
print(df.shape)
df.head()

No junk data anymore on `Order ID, Quanrtity Ordered, Order Date, Purchase Address`

In [ ]:
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

### Back to Duplicates

Check duplicates

In [ ]:
df[df.duplicated()] 

Sanity check for first row indicated in the previous table

In [ ]:
df.loc[873:877,:]

Drop duplicates

In [ ]:
df = df.drop_duplicates(keep='first').reset_index(drop=True)

print(df.shape)
df.head()

No duplicates anymore

In [ ]:
df[df.duplicated()] 

---

# More EDA and Feature Engineering

* Convert Data Type
* Feature engineering
  * Extract Information from date
  * Compute revenue
  * Add product line
  * Add synthetic cost and compute margin
  * Customer Address

### Convert Data Type

In [ ]:
df.info()

Convert
* Price and Quantity to number
* Order Date to datetime

In [ ]:
df['Price Each'] = df['Price Each'].astype(float)
df['Quantity Ordered'] = df['Quantity Ordered'].astype(int)
df['Order Date'] = pd.to_datetime(df['Order Date'])

df.info()

### Feature Engineering

#### Extract date information using `feature-engine`

In [ ]:
from feature_engine.datetime import DatetimeFeatures
pd.set_option('display.max_columns', None)
df = DatetimeFeatures(variables=['Order Date'], features_to_extract='all', drop_original=False).fit_transform(df)
df['Weekday'] =  df['Order Date'].dt.day_name()
df['YearMonth'] = df['Order Date'].dt.to_period('M')
df['YearQuarter'] = df['Order Date'].dt.to_period('Q')

# drop variables apparently not relevant for the project
df.drop(['Order Date_month_start', 'Order Date_month_end', 'Order Date_quarter_start',
         'Order Date_quarter_end', 'Order Date_year_start', 'Order Date_year_end',
         'Order Date_leap_year', 'Order Date_days_in_month', 'Order Date_second']
        ,axis=1, inplace=True)

print(df.shape)
df.head(2)

Check if all data is related to 2019
* Only 34 purchases in 2020

In [ ]:
df['Order Date_year'].value_counts()

Check which days of the month and hours were these 2020 purchases
* Day 1, from 0h to 5h
  * Maybe the customer took the decision to buy before midnight but bought after midnight? (Common dilemma in e-commerce) 
  * For the sake of assessment, we will consider purchases that were closed in 2019

In [ ]:
df.query('`Order Date_year` == 2020')[['Order Date_day_of_month','Order Date_hour' ]].value_counts()

For the sake of the assessment, we remove 2020

In [ ]:
df = df.query('`Order Date_year` == 2019')
print(df.shape)

Add flag for US Holidays


In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
holidays = USFederalHolidayCalendar().holidays(start='2019-01-01', end='2020-01-05')
holidays

In [ ]:
df['HolidayFlag'] = pd.to_datetime(df['Order Date'].dt.strftime('%Y-%m-%d')).apply(lambda x: True if x in holidays else False)

pd.DataFrame(data={"Count":df['HolidayFlag'].value_counts(),
                   "Relative %": round(df['HolidayFlag'].value_counts(normalize=True)*100 ,1)
                   })

#### Compute Revenue

In [ ]:
df['Revenue'] = df['Quantity Ordered'] * df['Price Each']

In [ ]:
df.head(3)

#### Add product line

In [ ]:
np.sort(df['Product'].unique()).tolist() # so I can copy and paste to create dictionary

Map Product and Product line

In [ ]:
dict_prod_line = {'20in Monitor': "PC and Video Games",
                  '27in 4K Gaming Monitor': "PC and Video Games",
                  '27in FHD Monitor': "PC and Video Games",
                  '34in Ultrawide Monitor': "PC and Video Games",
                  'AA Batteries (4-pack)': "Cable and Accessories",
                  'AAA Batteries (4-pack)': "Cable and Accessories",
                  'Apple Airpods Headphones': "Audio",
                  'Bose SoundSport Headphones': "Audio",
                  'Flatscreen TV': "TV",
                  'Google Phone': "Mobile",
                  'LG Dryer': "Household",
                  'LG Washing Machine': "Household",
                  'Lightning Charging Cable': "Cable and Accessories",
                  'Macbook Pro Laptop': "Computer and Laptop",
                  'ThinkPad Laptop': "Computer and Laptop",
                  'USB-C Charging Cable': "Cable and Accessories",
                  'Vareebadd Phone': "Mobile",
                  'Wired Headphones': "Audio",
                  'iPhone': "Mobile"
                  }
# dict_prod_line

In [ ]:
df['ProductLine'] = df['Product'].map(dict_prod_line)
df.head(2)

#### Add synthetic cost and compute margin


* **Reason** bring the use case to analyze margin
  * We will simplify that `Margin = Revenue - Cost`

Is the price constant for all products?
* Yes, since standard deviation is 0. 
* We will follow this pattern, so the individual product cost will be constant as well

In [ ]:
df.groupby(['Product'])['Price Each'].std()

What is the price for each product, colored by product line?
* Computer and Laptop, Mobile and Household tend to be more expensive

In [ ]:
plt.figure(figsize=(17, 4))
sns.barplot(data = df.groupby(['Product','ProductLine'])['Price Each'].median().sort_values(ascending=False).reset_index(),
            x = "Product", y = "Price Each", hue = "ProductLine", dodge = False)
plt.xticks(rotation=90)
plt.show()

Create product cost dictionary

In [ ]:
dict_prod_cost = {'20in Monitor': 50,
                  '27in 4K Gaming Monitor': 175,
                  '27in FHD Monitor': 70,
                  '34in Ultrawide Monitor': 180,
                  'AA Batteries (4-pack)': 1.2,
                  'AAA Batteries (4-pack)': 1.1,
                  'Apple Airpods Headphones': 45,
                  'Bose SoundSport Headphones': 40,
                  'Flatscreen TV': 170,
                  'Google Phone': 400,
                  'LG Dryer': 400,
                  'LG Washing Machine': 300,
                  'Lightning Charging Cable': 4,
                  'Macbook Pro Laptop': 500,
                  'ThinkPad Laptop': 350,
                  'USB-C Charging Cable': 3.8,
                  'Vareebadd Phone': 250,
                  'Wired Headphones': 4.5,
                  'iPhone': 300, 
                  }
# dict_prod_cost

Map cost based on product

In [ ]:
df['ProductCostEach'] = df['Product'].map(dict_prod_cost)
df['Cost'] = df['ProductCostEach'] * df['Quantity Ordered']
df.head(3)

Compute absolute margin (`Margin = Revenue - Cost`) and Percentual Margin

In [ ]:
df['Margin'] = df['Revenue'] - df['Cost']
df['PercentualMargin'] = round(df['Margin'] / df['Revenue'] * 100, 2)
df.head(2)

#### Customer Address

In [ ]:
 df[['Purchase Address']].head(2)

We are interested in taking US State Abbreviations (that happens after the last comma), and the city (happens in second last comma)

In [ ]:
df['State'] = df['Purchase Address'].apply(lambda x: x.split(",")[-1][1:3])
df['City'] = df['Purchase Address'].apply(lambda x: x.split(",")[-2][1:])
df['City'] = df['State'] + " , " + df['City']

print(df.shape)
df.head(2)

---

# Save processed data

* For simplicity, I ran the notebook on Colab, saved the data in the session, downloaded manually the data to my laptop, then updloaded to the GitHub Repo
* In the workplace, the notebook session would be connected directly to the repo, either running in a local IDE (ie: VS Code) or cloud IDE (ie.: Gitpod), and a more professional approach to persist the data be usedPersist data to inputs/dataset folder

In [ ]:
df.to_csv("kmart_processed_data.csv", index=False)

---

# Pipeline to process raw data

In the workplace, an application will fetch the raw data and a piece of code would process it, so the data can be used by subsequent tasks/applications

In [ ]:
print(df_raw.shape)
df_raw.head(3)

The data processes in the notebook include:
* Drop missing values
* Get rid of junk data
* Drop duplicated rows 
* Convert data type
* Extract information from order date
* Compute revenue
* Add product line
* Add cost and compute margin
* Extract information from customer address




The code below contains: 
* Custom transformers used to transform the data inside the pipeline
* Pipeline steps for data processing


**Note**
* the variables names and dictionaries would likely live in a config file
* for the purpose of the assessment, I hard coded here

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config
set_config(display="diagram") 
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
pd.set_option('display.max_columns', None)

from feature_engine.imputation import DropMissingData
from feature_engine.datetime import DatetimeFeatures

############################################################
#### Custom transformers to be added to the pipeline
############################################################
class GetRidJunkData(BaseEstimator, TransformerMixin):
  def __init__(self, variables):
    if not isinstance(variables, list): self.variables = [variables]
    else: self.variables = variables

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    for feature in self.variables:
      X = X.query(f'`{feature}` != "{feature}"')
    return X


class DropDuplicatedRows(BaseEstimator, TransformerMixin):
  def __init__(self): return None

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    X = X.drop_duplicates(keep='first').reset_index(drop=True)
    return X

class ConverDataType(BaseEstimator, TransformerMixin):
  def __init__(self): return None

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    X['Price Each'] = X['Price Each'].astype(float)
    X['Quantity Ordered'] = X['Quantity Ordered'].astype(int)
    X['Order Date'] = pd.to_datetime(X['Order Date'])
    return X


class ExtractInformationFromDate(BaseEstimator, TransformerMixin):
  def __init__(self, variables):
    if not isinstance(variables, list): self.variables = [variables]
    else: self.variables = variables

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    for feature in self.variables:
      X = DatetimeFeatures(variables=[feature], features_to_extract='all', drop_original=False).fit_transform(X)
      X['Weekday'] = X[feature].dt.day_name()
      X['YearMonth'] = X['Order Date'].dt.to_period('M')
      X['YearQuarter'] = X['Order Date'].dt.to_period('Q')
      
      X.drop([f'{feature}_month_start', f'{feature}_month_end', f'{feature}_quarter_start',
         f'{feature}_quarter_end', f'{feature}_year_start', f'{feature}_year_end',
         f'{feature}_leap_year', f'{feature}_days_in_month', f'{feature}_second']
        ,axis=1, inplace=True)
      
      X = X.query(f'`{feature}_year` == 2019')
      
      holidays = USFederalHolidayCalendar().holidays(start='2019-01-01', end='2020-01-05')
      X['HolidayFlag'] = pd.to_datetime(X[feature].dt.strftime('%Y-%m-%d')).apply(lambda x: True if x in holidays else False)

    return X


class ComputeRevenue(BaseEstimator, TransformerMixin):
  def __init__(self): return None

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    X['Revenue'] = X['Quantity Ordered'] * X['Price Each']
    return X


class AddProductLine(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.dict_prod_line = {'20in Monitor': "PC and Video Games",
                          '27in 4K Gaming Monitor': "PC and Video Games",
                          '27in FHD Monitor': "PC and Video Games",
                          '34in Ultrawide Monitor': "PC and Video Games",
                          'AA Batteries (4-pack)': "Cable and Accessories",
                          'AAA Batteries (4-pack)': "Cable and Accessories",
                          'Apple Airpods Headphones': "Audio",
                          'Bose SoundSport Headphones': "Audio",
                          'Flatscreen TV': "TV",
                          'Google Phone': "Mobile",
                          'LG Dryer': "Household",
                          'LG Washing Machine': "Household",
                          'Lightning Charging Cable': "Cable and Accessories",
                          'Macbook Pro Laptop': "Computer and Laptop",
                          'ThinkPad Laptop': "Computer and Laptop",
                          'USB-C Charging Cable': "Cable and Accessories",
                          'Vareebadd Phone': "Mobile",
                          'Wired Headphones': "Audio",
                          'iPhone': "Mobile"
                          }

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    X['ProductLine'] = X['Product'].map(self.dict_prod_line)
    return X


class AddCostAndComputeMargin(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.dict_prod_cost = {'20in Monitor': 50,
                          '27in 4K Gaming Monitor': 175,
                          '27in FHD Monitor': 70,
                          '34in Ultrawide Monitor': 180,
                          'AA Batteries (4-pack)': 1.2,
                          'AAA Batteries (4-pack)': 1.1,
                          'Apple Airpods Headphones': 45,
                          'Bose SoundSport Headphones': 40,
                          'Flatscreen TV': 170,
                          'Google Phone': 400,
                          'LG Dryer': 400,
                          'LG Washing Machine': 300,
                          'Lightning Charging Cable': 4,
                          'Macbook Pro Laptop': 500,
                          'ThinkPad Laptop': 350,
                          'USB-C Charging Cable': 3.8,
                          'Vareebadd Phone': 250,
                          'Wired Headphones': 4.5,
                          'iPhone': 300, 
                          }

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    X['ProductCostEach'] = X['Product'].map(self.dict_prod_cost)
    X['Cost'] = X['ProductCostEach'] * X['Quantity Ordered']
    X['Margin'] = X['Revenue'] - X['Cost']
    X['PercentualMargin'] = round(X['Margin'] / X['Revenue'] * 100, 2)
    return X


class ExtractAddressInformation(BaseEstimator, TransformerMixin):
  def __init__(self, variables):
    if not isinstance(variables, list): self.variables = [variables]
    else: self.variables = variables

  def fit(self, X, y=None): return self
      
  def transform(self, X):
    for feature in self.variables:
      X['State'] = X['Purchase Address'].apply(lambda x: x.split(",")[-1][1:3])
      X['City'] = X['Purchase Address'].apply(lambda x: x.split(",")[-2][1:])
      X['City'] = X['State'] + " , " + X['City']

    return X



############################################################
#### Pipeline for data processing
############################################################
def PipelineDataProcessing():
  pipeline_base = Pipeline([
      
      ("DropMissingData",DropMissingData() ),
      ("GetRidJunkData", GetRidJunkData(variables=['Order ID']) ),
      ("DropDuplicatedRows", DropDuplicatedRows() ),
      ("ConverDataType", ConverDataType()), 
      ("ExtractInformationFromDate", ExtractInformationFromDate(variables='Order Date')),
      ("ComputeRevenue", ComputeRevenue() ),
      ("AddProductLine", AddProductLine() ),
      ("AddCostAndComputeMargin", AddCostAndComputeMargin() ),
      ("ExtractAddressInformation", ExtractAddressInformation(variables= ['Purchase Address']))

    ])

  return pipeline_base

PipelineDataProcessing()

Transform the data based on the pipeline

In [ ]:
df_kmart = PipelineDataProcessing().fit_transform(df_raw)
print(df_kmart.shape)
df_kmart.head(3)

Sanity test: Is `df_kmart` the same as `df` (dataset we have been transforming thru the notebook)?

In [ ]:
df.equals(df_kmart)

---